In [26]:
import eazy.hdf5
import pandas as pd
import matplotlib.pyplot as plt

In [27]:

# We can generalise this the required field in ZFOURGE - Cosmos, CDFS, UDS 
# we need to begin by specififying the field to get the required data
field = 'COSMOS'

translate_file = {'CDFS': 'cdfs.v1.6.11.translate', 'COSMOS': 'cosmos.v1.3.6.translate', 'UDS': 'uds.v1.5.8.translate'}
param_file = {'CDFS': 'cdfs.v1.6.1.zphot.param', 'COSMOS': 'cosmos.v1.3.6.zphot.param', 'UDS': 'uds.v1.5.8.zphot.param'}
hdf5view_file = {'CDFS': 'cdfs.v1.6.1.h5', 'COSMOS': 'cosmos.v1.3.8.h5', 'UDS': 'uds.v1.5.10.h5'}


param_file = param_file[field]
translate_file = translate_file[field]
hdf5view_file = hdf5view_file[field]

In [28]:
self = eazy.photoz.PhotoZ(param_file=param_file, translate_file=translate_file, zeropoint_file=None,
                         load_prior=False, load_products=False)


Read default param file: cosmos.v1.3.6.zphot.param
Read CATALOG_FILE: cosmos.v1.3.8.cat
   >>> NOBJ = 20786
f_IRAC_80 e_IRAC_80 ( 21): IRAC/irac_tr4_2004-08-09.dat
f_IRAC_58 e_IRAC_58 ( 20): IRAC/irac_tr3_2004-08-09.dat
f_IRAC_45 e_IRAC_45 ( 19): IRAC/irac_tr2_2004-08-09.dat
f_IRAC_36 e_IRAC_36 ( 18): IRAC/irac_tr1_2004-08-09.dat
f_Ks e_Ks (255): FOURSTAR/Ks_cam_optics_sky.txt
f_J1 e_J1 (249): FOURSTAR/J1_cam_optics_sky.txt
f_J2 e_J2 (250): FOURSTAR/J2_cam_optics_sky.txt
f_J3 e_J3 (251): FOURSTAR/J3_cam_optics_sky.txt
f_Hs e_Hs (254): FOURSTAR/Hshort_cam_optics_sky.txt
f_Hl e_Hl (253): FOURSTAR/Hlong_cam_optics_sky.txt
f_Zp e_Zp ( 83): COSMOS/SUBARU_filter_z.txt
f_Z e_Z ( 92): megaprime/cfht_mega_z_cfh9801.dat
f_I e_I ( 91): megaprime/cfht_mega_i_cfh9701.dat
f_Rp e_Rp ( 81): COSMOS/SUBARU_filter_r.txt
f_R e_R ( 90): megaprime/cfht_mega_r_cfh9601.dat
f_G e_G ( 89): megaprime/cfht_mega_g_cfh9401.dat
f_V e_V ( 79): COSMOS/SUBARU_filter_V.txt
f_B e_B ( 78): COSMOS/SUBARU_filter_B.txt
f_U e

100%|██████████| 9/9 [04:11<00:00, 27.96s/it] 


Template   0: eazy_v1.1_sed1.dat (NZ=1).
Template   1: eazy_v1.1_sed2.dat (NZ=1).
Template   2: eazy_v1.1_sed3.dat (NZ=1).
Template   3: eazy_v1.1_sed4.dat (NZ=1).
Template   4: eazy_v1.1_sed5.dat (NZ=1).
Template   5: eazy_v1.1_sed6.dat (NZ=1).
Template   6: eazy_v1.1_sed7.dat (NZ=1).
Template   7: c09_del_8.6_z_0.019_chab_age09.40_av2.0.dat (NZ=1).
Template   8: erb2010_highEW.dat (NZ=1).
Process templates: 252.497 s


479it [00:19, 25.10it/s]


In [29]:
view = eazy.hdf5.Viewer(hdf5view_file)

h5: read eazy_v1.1_sed1.dat
h5: read eazy_v1.1_sed2.dat
h5: read eazy_v1.1_sed3.dat
h5: read eazy_v1.1_sed4.dat
h5: read eazy_v1.1_sed5.dat
h5: read eazy_v1.1_sed6.dat
h5: read eazy_v1.1_sed7.dat
h5: read c09_del_8.6_z_0.019_chab_age09.40_av2.0.dat
h5: read erb2010_highEW.dat


In [30]:
view.get_catalog()

id,ra,dec,z_spec,f_IRAC_80,e_IRAC_80,f_IRAC_58,e_IRAC_58,f_IRAC_45,e_IRAC_45,f_IRAC_36,e_IRAC_36,f_Ks,e_Ks,f_J1,e_J1,f_J2,e_J2,f_J3,e_J3,f_Hs,e_Hs,f_Hl,e_Hl,f_Zp,e_Zp,f_Z,e_Z,f_I,e_I,f_Rp,e_Rp,f_R,e_R,f_G,e_G,f_V,e_V,f_B,e_B,f_U,e_U,f_IA427,e_IA427,f_IA484,e_IA484,f_IA505,e_IA505,f_IA527,e_IA527,f_IA624,e_IA624,f_IA709,e_IA709,f_IA738,e_IA738,f_F125W,e_F125W,f_F160W,e_F160W,f_F140W,e_F140W,f_F606W,e_F606W,f_F814W,e_F814W,f_UVISTA_J,e_UVISTA_J,f_UVISTA_H,e_UVISTA_H,f_UVISTA_Ks,e_UVISTA_Ks,f_UVISTA_Y,e_UVISTA_Y,f_Ksall,e_Ksall
int64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
1,150.128601,2.174722,-99.00000,-15.862,11.253,-19.866,9.123,9.504,0.293,9.048,0.253,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,0.711,0.147,0.992,0.120,0.666,0.042,0.582,0.040,0.619,0.037,0.481,0.035,0.520,0.044,0.407,0.025,0.281,0.052,0.450,0.098,0.551,0.086,0.503,0.102,0.462,0.082,0.497,0.086,0.502,0.107,0.636,0.125,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,0.499,0.066,-99.000,-99.000,1.240,0.171,2.032,0.250,2.174,0.227,1.118,0.116,0.748,171697.000
2,150.160644,2.174749,-99.00000,-7.330,5.227,-1.584,4.437,0.681,0.138,0.646,0.132,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,0.146,0.119,0.270,0.097,0.187,0.034,0.062,0.033,0.072,0.030,0.022,0.029,0.030,0.036,0.002,0.020,0.010,0.042,0.003,0.080,0.038,0.070,-0.104,0.083,0.026,0.066,0.136,0.070,0.200,0.086,0.362,0.102,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,0.071,0.054,0.212,0.058,0.446,0.136,0.184,0.203,0.970,0.180,0.183,0.094,1.012,202858.812
3,150.122452,2.174830,-99.00000,-3.545,5.284,4.618,4.592,0.761,0.140,0.720,0.123,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,0.242,0.108,0.371,0.088,0.373,0.031,0.353,0.029,0.367,0.027,0.388,0.026,0.359,0.032,0.357,0.019,0.271,0.038,0.296,0.072,0.277,0.063,0.250,0.075,0.397,0.060,0.440,0.063,0.363,0.078,0.151,0.092,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,0.387,0.060,0.369,0.064,0.589,0.126,0.451,0.182,0.766,0.167,0.301,0.085,0.856,184789.500
4,150.070969,2.174860,-99.00000,0.565,5.565,7.941,4.369,8.041,0.140,14.256,0.158,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,3.060,0.127,2.720,0.105,1.893,0.036,0.896,0.035,0.875,0.032,0.296,0.031,0.575,0.039,0.185,0.022,0.005,0.044,0.085,0.086,0.172,0.074,0.272,0.088,0.569,0.071,0.894,0.074,1.209,0.091,2.110,0.108,1.715,0.052,4.441,0.066,-99.000,-99.000,0.699,0.068,2.493,0.076,4.997,0.151,4.752,0.218,3.985,0.195,4.016,0.101,1.251,251742.516
5,150.189209,2.174664,-99.00000,9.961,5.173,-1.307,4.311,6.251,0.151,9.549,0.144,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,3.485,0.122,3.629,0.099,3.023,0.035,1.772,0.034,1.771,0.030,1.396,0.029,1.288,0.037,1.124,0.021,1.333,0.043,1.094,0.083,1.195,0.071,1.357,0.084,1.393,0.068,1.475,0.071,2.715,0.088,2.865,0.105,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,1.798,0.055,3.419,0.062,4.982,0.138,5.058,0.207,5.927,0.181,3.929,0.094,2.004,260128.406
6,150.027451,2.174896,-99.00000,22.732,4.797,6.474,4.107,7.078,0.332,12.349,0.288,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,-99.000,2.303,0.115,2.587,0.092,2.245,0.032,1.498,0.030,1.888,0.027,1.536,0.027,1.641,0.034,1.244,0.020,0.814,0.038,1.312,0.076,1.539,0.066,1.557,0.077,1.587,0.0

In [31]:

# Get the redshifts, from the second column of the txt file
# read in the csv file
selected_ids_df = pd.read_csv('full_'+field+'_ids.csv', header=None)


# split into redshifts and ids, ignore the first row
selected_ids = selected_ids_df[0][1:]
selected_redshifts = selected_ids_df[1][1:]


In [32]:
object_ids = selected_ids
object_ids = [int(x) for x in object_ids]


In [33]:

# do the same with the redshifts
selected_redshifts = [float(x) for x in selected_redshifts]


In [34]:
len(object_ids)

7790

In [35]:
show_fnu = 0  # Adjust this as needed: 0 for erg/s/cm^2/Å, 1 for µJy, 2 for µJy/micron

In [36]:
z_id = 0
for obj_id in object_ids:
    # Call the function with get_spec=True for the current object ID
    sed_data = view.show_fit(id=obj_id, get_spec=True, show_fnu=show_fnu)

    # Extract 'templz' (wavelength) and 'templf' (flux density) from the data
    
    # observered wavelength
    wavelengths = sed_data['templz']
    
    # Convert the wavelength into restframe wavelength
    wavelengths = wavelengths / (1 + float(selected_redshifts[z_id]))
    z_id += 1
    
    # flux density
    flux_densities = sed_data['templf']

    # Create a DataFrame
    df_sed = pd.DataFrame({
        'Wavelength': wavelengths,
        'FluxDensity': flux_densities
    })

    # Define the flux unit for the filename and plot label
    flux_unit = '10^-19 erg/s/cm^2/Å' if show_fnu == 0 else ('µJy' if show_fnu == 1 else 'µJy/micron')
    
    # Save to CSV
    csv_file_name = f'SED_{field}_{obj_id}.csv'
    df_sed.to_csv("zfourgeSEDs/"+field+"/"+csv_file_name, index=False)

    # Plotting the SED in log-log space
    # plt.figure(figsize=(8, 5))
    # plt.loglog(df_sed['Wavelength'], df_sed['FluxDensity'], marker='o', linestyle='-', color='royalblue', label=f'SED for ID {obj_id}')
    # plt.xlabel('Wavelength (micron)')
    # plt.ylabel(f'Flux Density ({flux_unit})')
    # plt.title(f'Spectral Energy Distribution (SED) for ID {obj_id} - Log-Log Plot')
    # plt.legend()
    # plt.grid(True, which="both", ls="--")
    # plt.show()

c:\Users\Titan\Anaconda3\envs\eazy39\lib\site-packages\eazy\templates.py:680: RuntimeWarning: overflow encountered in multiply
  return (flam * self.wave**2 / (utils.CLIGHT*1.e10))
c:\Users\Titan\Anaconda3\envs\eazy39\lib\site-packages\eazy\templates.py:680: RuntimeWarning: overflow encountered in multiply
  return (flam * self.wave**2 / (utils.CLIGHT*1.e10))
c:\Users\Titan\Anaconda3\envs\eazy39\lib\site-packages\eazy\templates.py:680: RuntimeWarning: overflow encountered in multiply
  return (flam * self.wave**2 / (utils.CLIGHT*1.e10))
c:\Users\Titan\Anaconda3\envs\eazy39\lib\site-packages\eazy\templates.py:680: RuntimeWarning: overflow encountered in multiply
  return (flam * self.wave**2 / (utils.CLIGHT*1.e10))
c:\Users\Titan\Anaconda3\envs\eazy39\lib\site-packages\eazy\templates.py:680: RuntimeWarning: overflow encountered in multiply
  return (flam * self.wave**2 / (utils.CLIGHT*1.e10))
c:\Users\Titan\Anaconda3\envs\eazy39\lib\site-packages\eazy\templates.py:680: RuntimeWarning: o